<a href="https://colab.research.google.com/github/harmeet-singh-bagga/ExplainableNLPsuite/blob/main/ReadingGraphData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install opencv-python pillow pytesseract requests tesseract
!pip install pyforest
import pyforest

  Preparing metadata (setup.py) ... done
  Created wheel for pyforest: filename=pyforest-1.1.2-py2.py3-none-any.whl size=15900 sha256=1544e6c125b1295caec09e476fe1134b3447ff2ba1a5b4e031a72b5f2a7dc497
  Stored in directory: /root/.cache/pip/wheels/c5/ca/73/5cdc3d087111bfbdef90be5457aa03c00c0e32241b2752445c
Successfully built pyforest


In [9]:
import os
from PIL import Image

def load_chart_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            images.append(image_path)
    return images

# Example usage
folder_path = '/content/sample_data'  # Replace with your folder path
chart_images = load_chart_images(folder_path)
print(f"Loaded {len(chart_images)} images from {folder_path}.")


Loaded 9 images from /content/sample_data.


In [11]:
import cv2

def extract_bars_data(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Find contours to detect bars
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bar_data = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 10 and h > 10:  # Filter out small noise
            bar_data.append({"position": (x, y), "width": w, "height": h})
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imwrite('detected_bars.png', image)  # Save the image with detected bars
    return bar_data

# Example usage
image_path = '/content/sample_data/0a0d8dc51eff3dca2f90451f048ceb81_d3d3LmludGVyZmF4LmpwCTIwMi4yMzAuMTM3LjM=-0-0.png'  # Replace with your bar chart image path
bars = extract_bars_data(image_path)
print(f"Extracted bar data: {bars}")


Extracted bar data: [{'position': (0, 0), 'width': 629, 'height': 357}]


In [15]:
def extract_lines_data(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Detect lines using Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=10)
    line_data = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            line_data.append({"start": (x1, y1), "end": (x2, y2)})
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imwrite('detected_lines.png', image)  # Save the image with detected lines
    return line_data

# Example usage
image_path = '/content/sample_data/011c2152453331ddfd58060d76ac655d_d3d3LnRpbGx2YXh0YW5hbHlzLnNlCTE5My4xMy43NS4xNjc=.xls-3-0.png'
lines = extract_lines_data(image_path)
print(f"Extracted line data: {lines}")

<IPython.core.display.Javascript object>

Extracted line data: [{'start': (23, 179), 'end': (306, 179)}, {'start': (18, 23), 'end': (305, 23)}, {'start': (19, 25), 'end': (306, 25)}, {'start': (22, 125), 'end': (306, 125)}, {'start': (25, 74), 'end': (306, 74)}, {'start': (23, 127), 'end': (306, 127)}, {'start': (28, 228), 'end': (305, 228)}, {'start': (24, 76), 'end': (306, 76)}, {'start': (28, 230), 'end': (306, 230)}, {'start': (24, 177), 'end': (306, 177)}, {'start': (44, 256), 'end': (44, 24)}, {'start': (306, 232), 'end': (306, 26)}, {'start': (304, 257), 'end': (304, 26)}, {'start': (42, 257), 'end': (42, 26)}, {'start': (54, 306), 'end': (261, 306)}, {'start': (84, 279), 'end': (270, 279)}, {'start': (54, 280), 'end': (269, 283)}, {'start': (53, 309), 'end': (261, 312)}, {'start': (28, 232), 'end': (305, 232)}]


In [17]:
def extract_pie_slices_data(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)

    # Find contours to detect circular slices
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    slice_data = []

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        if len(approx) > 8:  # Circular shapes
            (x, y), radius = cv2.minEnclosingCircle(contour)
            slice_data.append({"center": (int(x), int(y)), "radius": int(radius)})
            cv2.circle(image, (int(x), int(y)), int(radius), (0, 255, 0), 2)

    cv2.imwrite('detected_pie_slices.png', image)  # Save the image with detected slices
    return slice_data

# Example usage
image_path = '/content/sample_data/0a0c983f2e69d6c45da004b071ffa8e7_d3d3LmFxdS5jYXQJODQuODguOTYuOTk=.xls-1-2.png'
slices = extract_pie_slices_data(image_path)
print(f"Extracted pie slice data: {slices}")

Extracted pie slice data: [{'center': (331, 26), 'radius': 14}]


In [18]:
import json

def convert_data_to_json(chart_data):
    return json.dumps(chart_data, indent=2)

# Example usage
chart_data = [{"position": (50, 150), "width": 100, "height": 200}]  # Example bar data
json_data = convert_data_to_json(chart_data)
print(f"Converted JSON: {json_data}")

Converted JSON: [
  {
    "position": [
      50,
      150
    ],
    "width": 100,
    "height": 200
  }
]


In [20]:
import requests

def get_gpt_insights(json_data, chart_type):
    gpt_api_url = "https://api.openai.com/v1/completions"
    headers = {
        "Authorization": f"Bearer YOUR_API_KEY",  # Replace with your OpenAI API key
        "Content-Type": "application/json"
    }

    prompt = f"Analyze the following {chart_type} chart data: {json_data}. Provide key insights."
    data = {
        "model": "gpt-3.5-turbo",
        "prompt": prompt,
        "max_tokens": 100
    }

    response = requests.post(gpt_api_url, headers=headers, json=data)

    if response.status_code == 200:
        return response.json()['choices'][0]['text']
    else:
        return "Failed to get insights."

# Example usage
chart_type = "bar"
json_data = '{"position": [50, 150], "width": 100, "height": 200}'
insights = get_gpt_insights(json_data, chart_type)
print(f"GPT Insights: {insights}")

GPT Insights: Failed to get insights.
